In [1]:
import datetime
import os
from mdagent import MDAgent

In [2]:
prompt21 = "Download and clean protein 1A3N."
llm_var = "gpt-4-turbo-2024-04-09"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools)
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",agent.llm.model_name,"\nTemperature: ",agent.llm.temperature)

date and time: 2024-09-09
time: 10:59:51
LLM:  gpt-4-turbo-2024-04-09 
Temperature:  0.1


In [3]:
agent.run(prompt21)

Thought: The task requires downloading and cleaning the protein with PDB ID 1A3N. I will first download the PDB file using the PDBFileDownloader tool and then clean it using the CleaningToolFunction.

Action:
```json
{
    "action": "PDBFileDownloader",
    "action_input": {
        "query": "1A3N"
    }
}
```PDB file found with this ID: 1A3N
Now that the PDB file for protein 1A3N has been successfully downloaded, the next step is to clean the file. I will use the CleaningToolFunction to remove heterogens, add missing atoms and hydrogens, replace nonstandard residues, and remove water. This will ensure the protein structure is suitable for further analysis or simulation.

Action:
```json
{
    "action": "CleaningToolFunction",
    "action_input": {
        "pdb_id": "1A3N_105959",
        "replace_nonstandard_residues": true,
        "add_missing_atoms": true,
        "remove_heterogens": true,
        "remove_water": true,
        "add_hydrogens": true,
        "add_hydrogens_ph": 7.0

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [4]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
registry = agent.path_registry
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

date and time: 2024-09-09
time: 11:00:38
Files found in registry: 1A3N_105959: PDB file downloaded from RSCB
 PDBFile ID: 1A3N_105959
 1A3N_110034: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 


In [6]:
path_before = registry.get_mapped_path("1A3N_105959")
path_after = registry.get_mapped_path("1A3N_110034")
assert os.path.exists(path_before)
assert os.path.exists(path_after)
assert path_before != path_after

In [7]:
import mdtraj as md
raw_file = md.load(path_before)
clean_file = md.load(path_after)

In [8]:
#residues before
residues_before = set([str(res)[:3] for res in raw_file.top.residues])
residues_after = set([str(res)[:3] for res in clean_file.top.residues])
print("Residues before:",residues_before)
print("Residues after:",residues_after)
print("Residues removed:",residues_before - residues_after)

Residues before: {'LYS', 'HOH', 'TRP', 'LEU', 'HIS', 'GLN', 'ALA', 'GLY', 'ASN', 'SER', 'THR', 'GLU', 'HEM', 'PHE', 'ARG', 'TYR', 'ASP', 'CYS', 'MET', 'PRO', 'VAL'}
Residues after: {'LYS', 'TRP', 'LEU', 'HIS', 'GLN', 'ALA', 'GLY', 'ASN', 'SER', 'THR', 'GLU', 'PHE', 'ARG', 'TYR', 'ASP', 'CYS', 'MET', 'PRO', 'VAL'}
Residues removed: {'HOH', 'HEM'}


# Experiment Result:
### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ✅
### No hallucination ✅
### Logic make sense ✅
### Correct Answer  ✅
### Number of steps completed: 2
### % steps completed: 100%